# Wprowadzenie do używania HuggingFace

Instalacja bibliotek HF

In [ ]:
!pip3 install transformers[sentencepiece]==4.18.0
!pip3 install datasets==1.15.1
!pip3 install huggingface_hub>=0.1.0,<1.0.0

## Pipelines

Na najwyższym poziomie abstrakcji znajduje się pipeline() - jak wskazuje nazwa jest to gotowy pipeline, pod którym kryje się tokenizer i model, oraz cały związany z tym pre- i postprocessing. 

In [ ]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis")
pipe(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

Inne gotowe implementacje, które możemy przetestować to m.in.:
* feature-extraction - wektoryzacja tekstu
* fill-mask - uzupełnianie ukrytych słów (oznaczonych przez \<mask\>)
* ner (named entity recognition) - rozpoznanie 'bytów' w tekście
* question-answering - odpowiadanie na pytania
* sentiment-analysis - analiza (klasyfikacja) sentymentu
* summarization - podsumowanie tekstu
* text-generation - generowanie tekstu
* translation - tłumaczenie 
* zero-shot-classification - klasyfikacja zero-shot

---

Jeśli przyjrzymy się wynikom wykonanego przez nas wcześniej kodu, pipeline() informuje nas o tym, że skoro nie podaliśmy wprost konkretnego modelu, użyty zostanie pewien z góry ustalony model domyślny. 

Poniżej ponownie stworzymy pipeline dla analizy sentymentu z tym samym modelem (distilbert-base-uncased-finetuned-sst-2-english), ale tym razem wprost zainicjalizujemy model oraz odpowiadający mu tokenizer.

Do inicjalizacji odpowiedniego modelu możemy użyć [AutoTokenizer](https://huggingface.co/docs/transformers/main/en/autoclass_tutorial#autotokenizer) oraz [AutoModelForSequenceClassification](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModelForSequenceClassification).

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint_name = 'distilbert-base-uncased-finetuned-sst-2-english'

tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_name)

pipe = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)
pipe(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

Osiągneliśmy te same wyniki co ostatnio, więc już wszystko się zgadza. 

Gdy spróbujemy użyć po prostu [AutoModel](https://huggingface.co/docs/transformers/v4.18.0/en/model_doc/auto#auto-classes):

In [ ]:

from transformers import AutoModel

checkpoint_name = 'distilbert-base-uncased-finetuned-sst-2-english'

tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
model = AutoModel.from_pretrained(checkpoint_name)

pipe = pipeline("sentiment-analysis", tokenizer=tokenizer, model=model)
pipe(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

Dlaczego otrzymaliśmy błąd? Jeśli przyjrzymy się pierwszej linijce, możemy zauważyć ostrzeżenie o tym, że część wag jest ignorowana - a konkretnie wagi klasyfikatora, który interpretuje logits, które są wyjściem transformerów. 

AutoModel zakłada, że gdy chcemy wczytać model, chodzi nam tylko i wyłącznie o enkodery/dekodery, ignorowane są jakiekolwiek 'głowy'. 
Aby wczytać także wagi klasyfikatora, musimy być bardziej precyzyjni i użyć odpowiedniego AutoModel dla danego zadania.


---



W formie prostego zadania, spróbuj na podstawie tego co zostało już przedstawione, stworzyć pipeline do tłumaczenia tekstu z polskiego na angielski. 

Podpowiedź - wyszukaj odpowiedni checkpoint na [HuggingFace](https://huggingface.co/models) oraz klasę [AutoModel](https://huggingface.co/docs/transformers/main/en/model_doc/auto#transformers.AutoModel) dla tego typu zadań (seq2seq).

In [ ]:
# Rozwiązanie:
from transformers import AutoModelForSeq2SeqLM

checkpoint_name = 'Helsinki-NLP/opus-mt-pl-en'
tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_name)

pipe = pipeline("translation", tokenizer=tokenizer, model=model)
pipe(
    [
        "Losowy tekst po polsku",
        "Drugi tekst po polsku",
    ]
)

--- 

Teraz jest dobry moment, żeby dokładniej się przyjrzeć temu, co się dzieje między wejściem tekstu, a wyjściem modelu.

![](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)

Tak przedstawia się ogólny zarys, działanie modelu powinniśmy rozumieć, czas natomiast przyjrzeć się temu jak zamieniamy tekst w wejście do modelu.

## Tokenizacja
Tokenizacja odpowiada za zmianę tekstu (który mogą zrozumieć ludzie) w liczby (które może zrozumieć komputer). 

Wyróżniane są głównie 3 sposoby tokenizacji:


---


**Na poziomie słów**\
Działa dobrze, ale efektem ubocznym są ogromne słowniki - każde słowo musi mieć swoje własne ID, jest to jeszcze bardziej problematyczne w polskim, gdzie każda odmiana pewnego słowa byłaby dodatkowym wejściem (możnaby użyć lematyzacji, ale to także ma swoje wady). Dodatkowo każde słowo z poza słownika, musi być zastąpione tokenem \<unk\>.


---


**Na poziomie znaków**\
Eliminuje problem z rozmiarem słownika, znacznie zredukuje także częstość występowania \<unk\>. Tutaj natomiast problemem jest to, że znacznie rośnie ilość tokenów, które musi przetworzyć model, dodatkowo pojedyncze tokeny mają znacznie mniejsze "znaczenie", jako że reprezentują znaki zamiast słów.


---


**Na poziomie podsłów**\
Rozwiązanie pośrednie pomiędzy podziałem na słowa, a na znaki, mające łączyć zalety obu sposobów - polega na podziale słów na pomniejsze fragmenty. Obecnie najczęściej używany sposób, istnieje kilka sposobów podziału na podsłowa. 


---

Przykład jak podzielone zostało by "Zdanie do tokenizacji" tymi 3 sposobami:\
![](https://drive.google.com/uc?id=1mn4BiH9IkrfdvMl-7n1h7J4ITj3rcIds)



In [ ]:
text = "Przetwarzanie języka naturalnego to interdyscyplinarna dziedzina łącząca zagadnienia sztucznej inteligencji i językoznawstwa"

tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")

print("Tokenizacja na poziomie słow: ", text.split())
print("Tokenizacja na poziomie znaków: ", [char for char in text.replace(' ', '')])
print("Tokenizacja na poziomie podsłów (konkretnie byte-pair encoding): ", tokenizer.tokenize(text))

Gdy tekst jest już podzielony na tokeny, można przy pomocy słownika tokenizera zmienić je na ID.

In [ ]:
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

In [ ]:
tokenizer.vocab

Te ID, w teorii mogą już być wysłane do modelu po owinięciu ich w tensor odpowiedniego frameworku (w naszym przypadku PyTorch).

Po wcześniejszych eksperymentach, jeszcze raz przypisujemy model i tokenizator - w przypadku pracy z wieloma modelami należy pamiętać żeby **zawsze używać tokenizatora odpowiadającego modelowi!**

Między niektórymi modelami teoretycznie może się zdarzyć, że tokenizatory będą kompatybilne, ale w większości przypadków przez różnice w słowniku napotkamy wyjątek, lub przez to że pod jednym ID tokenizator i model mają zupełnie inny token, zwracane przez model wyjście będzie bezsensu.

In [ ]:
import torch

checkpoint_name = 'distilbert-base-uncased-finetuned-sst-2-english'

tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_name)

tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
input_ids = torch.tensor([ids])

output = model(input_ids)
print("Logits:", output.logits)

Dla pewności, możemy także z powrotem zmienić ID na tekst, jest to też przydatne, gdy nasz model będzie miał generować tekst.

In [ ]:
tokenizer.decode(ids)

Jednak, gdy tym sposobem jak wcześniej spróbujemy przekazać do modelu na raz kilka zdań, napotkamy błąd - w praktyce są jeszcze dwie ważne rzeczy, którymi zajmują się tokenizatory:
- truncation - przycięcie zbyt długich wektorów tokenów, do maksymalnej akceptowanej przez model
- padding - przy wielu tekstach (batch), wyrównujemy wszystkie tensory z tokenami do nadłuższego w danym batchu, poprzez dodawanie specjalnego tokenu (\<pad\>, 0, ...) do krótszych tensorów

Ze względu na atencję w transformerach, aby w trakcie liczenia atencji nie były brane pod uwagę tokeny paddingu, tokenizatory zwracają także *attention_mask*, określające czy dany token powinien być wliczany przy atencji.

Teraz zrozumiałe powinny być wartości zwracane przez tokenizer:

In [ ]:
texts = [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]

tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
print(tokens)

Dodatkowo, przy niektórych modelach wstawiane mogą być dodatkowe tokeny, przykładowo BERT przy zadaniach klasyfikacji, oczekuje na początku wejścia specjalny token [CLS] oraz na koniec każdego tekstu [SEP], pozwalający mu lepiej zrozumieć koniec jednego wejścia i początek drugiego.  

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
encoded_input = tokenizer("Do We Have Any Mathematical Proof That Pi Is Infinite?"," Does pi go on indefinitely?", return_tensors='pt')
tokenizer.decode(encoded_input["input_ids"][0])

## Model
Gdy doszliśmy już do tego co otrzymuje model, możemy przyjrzeć się dokładniej co znajduje się na jego wyjściu

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
model(encoded_input['input_ids'])

In [ ]:
model = AutoModel.from_pretrained("bert-base-cased")

In [ ]:
output = model(encoded_input['input_ids'])
output['last_hidden_state'].shape

# Trening modeli
Wiedząc już dokładniej co robi tokenizer, możemy przejść do trenowania modeli - przeprowadzimy prosty fine-tuning.

---

Dalszy przykład będzie na podstawie zbioru danych [glue](https://huggingface.co/datasets/glue) - jest to jeden z popularniejszych zbiorów NLP, z podzbiorami do wielu różnych zadań. Skupimy się na podzbiorze *qqp* - jest to zbiór par pytań z serwisu Quora, wraz z oznaczeniem czy dane pytania są semantycznie takie same.
Zatem klasyfikator wytrenowany na tym zbiorze, będzie w stanie stwierdzić czy podane mu dwa pytania, pytają się o to samo.

Jako model użyjemy do tego [BERTa](https://huggingface.co/bert-base-uncased).

---

Tu kolejną bardzo przydatną funkcjonalnością HuggingFace, jest ich biblioteka *datasets*, która pozwala w prosty i szybki sposób pobrać i przygotować dane do treningu.

## Zbiór danych i tokenizacja


In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("glue", "qqp")

HuggingFace zaimplementowało *datasets*, w taki sposób że w pamięci RAM trzymane są tylko te przypadki, które potrzebujemy, a reszta jest trzymana na dysku. Zwalnia nas to z własnoręcznego zarządzania pamięcią, która często bywa konieczna, przy tym jakie rozmiary mają niektóre zbiory w NLP.

In [ ]:
raw_dataset

Na potrzeby warsztatu weźmiemy pod uwagę tylko 10% zbioru

In [ ]:
raw_dataset['train'] = raw_dataset['train'].shard(num_shards=100, index=0)
raw_dataset['test'] = raw_dataset['test'].shard(num_shards=100, index=0)
raw_dataset['validation'] = raw_dataset['validation'].shard(num_shards=100, index=0)

Jak widzimy gotowy jest już podział na zbiór treningowy, testowy i walidacyjny. 

Możemy jeszcze się przyjrzeć jak wygląda pojedynczy przypadek:

In [ ]:
raw_dataset['train'].features


In [ ]:
raw_dataset['train'][0]

Dane te trzeba jeszcze przygotować, zanim będziemy mogli użyć je do treningu, gdy spojrzymy na kartę modelu, możemy zauważyć, że jako wejście powinniśmy dawać do modelu:\
[CLS] \<question1\> [SEP] \<question2\>

Jak było widoczne na przykładzie wcześniej, jest to już coś co obsługuje tokenizer modelu:


In [ ]:
checkpoint = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
sample = raw_dataset['train'][0]
tokenizer.decode(tokenizer(sample['question1'], sample['question2'])['input_ids'])

Musimy zatem w ten sposób przetworzyć cały zbiór, aby dalej korzystać z ułatwień *datasets*, zamiast iterować i tworzyć własny zbiór z zwracanych przez tokenizer dictów, możemy użyć [map()](https://huggingface.co/docs/datasets/v2.1.0/en/package_reference/main_classes#datasets.Dataset.map) po zdefiniowany naszej tokenizacji w formie funkcji. 

In [ ]:
def tokenize(sample):
  return tokenizer(sample['question1'], sample['question2'], truncation=True)

tokenized_datasets = raw_dataset.map(tokenize)
tokenized_datasets

In [ ]:
tokenizer.decode( tokenized_datasets['train'][0]['input_ids'] )

## Padding

Została jeszcze jedna kwestia - padding. 
Możemy dodać padding bezpośrednio do zbioru, razem z tokenizacją w ten sposób:

```
tokenized_datasets = raw_dataset.map(tokenize, padding=True)
```
Rozwiązanie tego w taki sposób ma jednak jeden problem - wszystkie przykłady w zbiorze będą miały dodany padding, tak aby były równe najdłuższemu przykładowi, co w zależności od rozłożenia naszych danych, może mieć znanczny wpływ na prędkość treningu. 

Lepszym od tego sposobem jest dynamic padding - jest to dodawanie paddingu indywidualnie do każdego batcha, dzięki czemu przykłady nie są długości najdłuższego przypadku w całym zbiorze, a jedynie w konkretnym batchu.

--- 

Funkcja składająca przypadki w batch nazywa się collate, która jest przekazywana do DataLoader'a. 
Domyślnie funkcja ta zajmuje się jedynie zamianą przypadków w tensory oraz ich konkatenacją, ale zamiast pisania jej ręcznie, możemy użyć [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorWithPadding) z biblioteki *transformers*.

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Najpierw tworzymy obiekt przechowujący argumenty treningowe

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("bert-finetuned-qqp")

In [ ]:
training_args

Inicjalizacja modelu do klasyfikacji.

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

## Trening przy pomocy klasy Trainer

Poniżej tworzymy obiekt klasy Trainer. Zajmuje on się m.in. włączaniem i wyłączaniem gradientów, używaniem Data Loader'ów dla zbiorów treningowych/testowych/walidacyjnych, przenoszeniem odpowiednich danych z CPU na GPU i z powrotem, logowaniem np. do tensorboarda itd., ma też dodatkowe funkcjonalności jak szukanie odpowiedniego learning rate.
Eliminuje on więc potrzebę pisania kodu treningu, który w zasadzie w większości przypadków wygląda tak samo.

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Ewaluacja:

In [ ]:
import numpy as np
from datasets import load_metric

predictions = trainer.predict(tokenized_datasets["validation"])

preds = np.argmax(predictions.predictions, axis=-1)

metric = load_metric("glue", "qqp")
metric.compute(predictions=preds, references=predictions.label_ids)

Wynik będzie się u każdego różnić, ale model powinnien mieć około 81% celności, co jak na rozmiar użytego zbioru danych i czas treningu, jest nienajgorszy.

Możemy spróbować użyć modelu:

In [ ]:
sample = raw_dataset['validation'][0]
question_1 = sample['question1']
question_2 = sample['question2']

print(question_1, '\n', question_2)

preds = trainer.predict([tokenizer(question_1, question_2)]).predictions[0]

if preds[0] > preds[1]:
  print("Model predicted that these questions are different.")
else:
  print("Model predicted that these questions are about the same thing.")

Podsumowanie całego dotychczasowego kodu do fine-tuningu:
```
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer

def tokenize(sample):
  return tokenizer(sample['question1'], sample['question2'], truncation=True)

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

raw_dataset = load_dataset("glue", "qqp")
tokenized_datasets = raw_dataset.map(tokenize)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments("bert-finetuned-qqp")
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()
```

## Ręczny trening
Jeśli jednak nie chcemy wykorzystywać klasy Trainer, bez problemu można samemu przeprowadzić trening, co da nam większią kontrolę.
Żeby się nie pogubić, przeprowadzimy całą inicjalizację od nowa.


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer

def tokenize(sample):
  return tokenizer(sample['question1'], sample['question2'], truncation=True)

checkpoint = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

raw_dataset = load_dataset("glue", "qqp")
raw_dataset['train'] = raw_dataset['train'].shard(num_shards=50, index=0)
raw_dataset['test'] = raw_dataset['test'].shard(num_shards=50, index=0)
raw_dataset['validation'] = raw_dataset['validation'].shard(num_shards=10, index=0)
tokenized_datasets = raw_dataset.map(tokenize)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
print(tokenized_datasets)

Najważniejsze rzeczy, które wykonywał Trainer, którymi trzeba się zająć to:
* zostawić w Dataset tylko te kolumny, których spodziewa się model (w tym wypadku trzeba usunąć 'question1', 'question2', 'idx', zmienić 'label' na 'labels')
* stworzyć DataLoadery, zwracające tensory
* zainicjalizować optimizer i  learning rate scheduler.
* przenoszenie zmiennych z CPU na GPU i wzajemnie
* pętla treningowa i ewaluacji


In [ ]:
tokenized_datasets["train"].column_names

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(['question1', 'question2', 'idx'])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

Przygotowanie DataLoader'ów

In [ ]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)

Możemy od razu się upewnić że wszystko działa w poniższy sposób:

In [ ]:
for batch in train_dataloader:
    break
print({k: v.shape for k, v in batch.items()})   # Tutaj przy okazji można zauważyć, że rozmiar tensorów będzie różny za każdym razem - jest to efekt dynamic padding
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

Używanie CUDA jeśli jest dostępna

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

W tym momencie, tak wyglądałaby pętla treningu:

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

A tak ewaluacji:

In [ ]:
from datasets import load_metric

metric = load_metric("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

# Udostępnianie modeli
Możemy też bez problemu udostępniać swoje modele na HuggingFace.

Jeśli zamierzamy wytrenować model przy pomocy Trainer, wystarczy dodatkowo przekazać argument *push_to_hub=True* do TrainingArguments, aby model po zakończonym treningu został spushowany do naszego repozytorium.

W innym wypadku, aby umieścić model i jego tokenizer w repozytorium, wystarczy zrobić to:
```
model.push_to_hub("model-name")
tokenizer.push_to_hub("model-name")
```

Przed tym trzeba się jeszcze zalogować do HuggingFace, z poziomu notatnika można to zrobić w ten sposób:
```
from huggingface_hub import notebook_login

notebook_login()
```

### Zadanie

Na podstawie powyższego przykładu, spróbuj wykonać fine-tuning dla jakiegoś polskiego zadania. 
Podobnym przykładem byłby fine-tuning [HerBERTa](https://huggingface.co/allegro/herbert-base-cased) na zbiorze [PSC](https://huggingface.co/datasets/psc) - otrzymalibyśmy klasyfikator oparty na BERT, który sprawdzałby czy jeden z podanych tekstów jest podsumowaniem drugiego.

Pamiętaj, że należy zwrócić uwagę czy checkpoint modelu, który bierzemy był wytrenowany albo na polskim zbiorze, lub przynajmniej wielojęzycznym, zawierającym polski.

In [ ]:
# Rozwiązanie
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer

def tokenize(sample):
  return tokenizer(sample['extract_text'], sample['summary_text'], truncation=True)

checkpoint = 'allegro/herbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

raw_dataset = load_dataset("psc", "default")

raw_dataset['train'] = raw_dataset['train'].shard(num_shards=10, index=0)
raw_dataset['test'] = raw_dataset['test'].shard(num_shards=10, index=0)

tokenized_datasets = raw_dataset.map(tokenize)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

training_args = TrainingArguments("test-trainer",per_device_eval_batch_size=4, per_device_train_batch_size=4)
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
sample = raw_dataset['test'][9]
question_1 = sample['extract_text']
question_2 = sample['summary_text']
label = sample['label']

print(question_1, '\n', question_2, '\n', label)
print(tokenizer.decode(tokenizer(question_1, question_2)['input_ids']))
preds = trainer.predict([tokenizer(question_1, question_2)]).predictions[0]

if preds[0] > preds[1]:
  print("Model predicted that these are different.")
else:
  print("Model predicted that these are about the same thing.")